In [17]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

hero_data = pd.read_csv('./data/heroCombination.csv')

hero_id = pd.read_csv('./data/heroList.csv')
type_lst = pd.DataFrame([{1:'边',2:'中',3:'射',4:'野',5:'辅'}]).T
hero_id['position'] = hero_id['type_1'].apply(lambda x: type_lst.loc[x])
# hero_lst = hero_id[['heroId','heroName']].set_index('heroId')
hero_lst = hero_id[['heroId','heroName','position']].set_index('heroId')

Win_rate_matrix_adj = pd.read_excel('./result/Win_rate_matrix_adj.xlsx',index_col=0)
Win_rate_ally_matrix_adj = pd.read_excel('./result/Win_rate_ally_matrix_adj.xlsx',index_col=0)

Win_rate_matrix = pd.read_excel('./result/Win_rate_matrix.xlsx',index_col=0)
Win_rate_ally_matrix = pd.read_excel('./result/Win_rate_ally_matrix.xlsx',index_col=0)

Win_matrix_ally_rank = pd.read_excel('./result/Win_matrix_ally_rank.xlsx',index_col=0)
Win_matrix_rank = pd.read_excel('./result/Win_matrix_rank.xlsx',index_col=0)

Hero_matrix_S38 = pd.read_excel('./result/Hero_matrix_S38.xlsx',index_col=0)

In [18]:
pos_lst = ['韩信','裴擒虎','影','坦元','诸葛亮','镜','司马懿','典韦','澜','暃','马超','宫本武藏']
# Win_matrix =  Win_rate_matrix_adj.loc[pos_lst]
# Win_matrix_ally =  Win_rate_ally_matrix_adj.loc[pos_lst]

# Win_matrix_ally_rank = Win_matrix_ally.rank(ascending=True,axis=0)
# Win_matrix_rank = Win_matrix.rank(ascending=True,axis=0)
# # Win_matrix_ally_rank.to_excel('./result/Win_matrix_ally_rank.xlsx')
# # Win_matrix_rank.to_excel('./result/Win_matrix_rank.xlsx')

# hero_lst.query("position == '射'")['heroName'].values

In [21]:
def hero_commendation_score(Win_matrix_ally_rank = None, Win_matrix_rank = None, Hero_matrix = None, allys=['关羽','王昭君','张良'], enemys=['马可波罗','瑶'], pos='野', adj=True):
    
    enemy_weight = {'边': 0.2,'野': 0.25,'中': 0.15,'射': 0.3,'辅': 0.1}
    ally_weight = {'边': 0.30,'野': 0.0,'中': 0.25,'射': 0.25,'辅': 0.20}

    allys = list(set(allys).intersection(set(hero_lst['heroName'].values)))
    enemys = list(set(enemys).intersection(set(hero_lst['heroName'].values)))

    # pos_lst = hero_lst[hero_lst['position'] == pos]['heroName'].values
    pos_lst = ['韩信','裴擒虎','影','坦元','诸葛亮','镜','司马懿','典韦','澜','暃','马超','宫本武藏']
    
    # RankIC 调整：
    if Win_matrix_ally_rank is None:
        if adj:
            Win_matrix = Win_rate_matrix_adj.loc[pos_lst]
            Win_matrix_ally = Win_rate_ally_matrix_adj.loc[pos_lst]
        else:
            Win_matrix = Win_rate_matrix.loc[pos_lst]
            Win_matrix_ally = Win_rate_ally_matrix.loc[pos_lst]
        Win_matrix_ally_rank = Win_matrix_ally.rank(ascending=True,axis=0)
    
    if Win_matrix_rank is None:
        if adj:
            Win_matrix = Win_rate_matrix_adj.loc[pos_lst]
            Win_matrix_ally = Win_rate_ally_matrix_adj.loc[pos_lst]
        else:
            Win_matrix = Win_rate_matrix.loc[pos_lst]
            Win_matrix_ally = Win_rate_ally_matrix.loc[pos_lst]
        Win_matrix_rank = Win_matrix.rank(ascending=True,axis=0)
    
    if Hero_matrix is None:
        Hero_matrix = pd.DataFrame()
        
    res_enemy = pd.DataFrame()
    for enemy in enemys:
        # print(enemy)
        enemy_pos = hero_lst.query(f"heroName == '{enemy}'")['position'].values[0]
        weight = enemy_weight[enemy_pos]
        res_enemy = pd.concat([res_enemy, Win_matrix_rank[enemy]*weight],axis=1)

    res_ally = pd.DataFrame()
    for ally in allys:
        ally_pos = hero_lst.query(f"heroName == '{ally}'")['position'].values[0]
        weight = ally_weight[ally_pos]
        res_ally = pd.concat([res_enemy, Win_matrix_ally_rank[ally]*weight],axis=1)

    res_ally = res_ally * 0.50
    res_enemy = res_enemy * 1.00
    Hero_matrix = Hero_matrix * 0.10

    res = pd.concat([res_ally,res_enemy,Hero_matrix],axis=1,keys = ['Ally','Enemy','Extra'],names=['modules','attrs'])
    res['Final_Score'] = res.sum(1)
    
    res = res.sort_values(by = 'Final_Score',ascending=False).head(8)
    return res

In [22]:
hero_commendation_score( Win_matrix_ally_rank = Win_matrix_ally_rank, Win_matrix_rank = Win_matrix_rank, Hero_matrix = Hero_matrix_S38,
    allys = [], 
    enemys=['马可波罗','瑶'], adj=True
)

modules Enemy      Extra                          Final_Score
attrs    马可波罗    瑶    皮肤   胜率   梯度   冲分   熟练   金牌            
裴擒虎       4.2  0.6   1.0  0.6  0.7  0.9  1.0  0.4         9.4
镜         4.5  0.8   0.8  0.3  1.1  0.5  0.9  0.3         9.2
诸葛亮       3.0  0.9   0.9  0.8  0.3  0.7  0.7  0.5         7.8
韩信        2.1  0.5   1.1  0.9  0.9  0.3  1.1  0.9         7.8
马超        3.9  1.5   0.6  0.1  1.0  0.1  0.2  0.1         7.5
司马懿       2.4  1.0   0.4  0.4  0.1  1.0  0.5  0.7         6.5
典韦        1.8  1.2   0.5  1.0  0.6  0.4  0.6  0.2         6.3
坦元        0.9  0.7   0.1  0.7  0.2  1.1  0.3  1.1         5.1

In [89]:
Win_rate_matrix_adj.rank().loc['镜'].sort_values(ascending=False)

鬼谷子     122.0
西施      121.0
成吉思汗    121.0
盾山      120.0
橘右京     120.0
        ...  
莱西奥       8.0
坦元        4.0
程咬金       3.0
海月        2.0
亚瑟        1.0
Name: 镜, Length: 122, dtype: float64

In [41]:
Win_rate_matrix_adj.loc['镜'].sort_values(ascending=False)

刘邦      0.512043
大司命     0.511985
弈星      0.511913
马可波罗    0.511604
亚连      0.511494
          ...   
雅典娜     0.495100
公孙离     0.494957
嫦娥      0.493949
戈娅      0.492943
海月      0.489940
Name: 镜, Length: 122, dtype: float64